In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# hyperparameters
num_heads = 20
n_layer = 20
dropout = 0.2
n_embd = 100

batch_size = 1 # number of independent tasks to be processed in parallel
block_size = 1000 # number of points to be processed per task # real block size is twice that value since xs and ys are interleaved 

max_iters = 6000
learning_rate = 3e-4

eval_interval = 500
eval_iters = 200


# data parameters
n_dims = 10
n_points = 10000
n_tasks = 1

assert n_embd > num_heads
assert block_size < n_points


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [3]:
# xs and ys (padded with 0s) interleaved 
data_train = torch.zeros(n_tasks, 2*n_points, n_dims)
data_train[:,::2,:] = torch.randn(n_tasks, n_points, n_dims)
tasks_data_train = torch.randn(n_tasks, n_dims)
data_train[:,1::2,:1] = data_train[:,::2,:] @ tasks_data_train.unsqueeze(-1)
data_train = data_train.to(device)

data_val = torch.zeros(n_tasks, 2*n_points, n_dims)
data_val[:,::2,:] = torch.randn(n_tasks, n_points, n_dims)
tasks_data_val = torch.randn(n_tasks, n_dims)
data_val[:,1::2,:1] = data_val[:,::2,:] @ tasks_data_val.unsqueeze(-1)
data_val = data_val.to(device)

In [4]:
def get_batch(split):
    data = data_train if split == 'train' else data_val
    idx_tasks = torch.randint(0, n_tasks, (batch_size,))
    idx_points = torch.randperm(n_points)[:block_size]*2
    dummy_tensor = torch.zeros(2*n_points)
    dummy_tensor[idx_points] = 1
    dummy_tensor[idx_points+1] = 1
    idx_points = torch.where(dummy_tensor)[0]
    return data[idx_tasks][:, idx_points].to(device)

In [6]:
class Head(nn.Module):
    # one head of self-attention
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(2*block_size, 2*block_size))) # used for masking
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        # x: batch_size x 2*block_size x n_embd
        k = self.key(x) # batch_size x 2*block_size x head_size
        q = self.query(x) # batch_size x 2*block_size x head_size
        # compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # batch_size x 2*block_size x 2*block_size
        wei = wei.masked_fill(self.tril == 0, float('-inf')) # batch_size x 2*block_size x 2*block_size
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # apply attention (weighted aggregation) to values
        v = self.value(x) # batch_size x 2*block_size x head_size
        out = wei @ v # batch_size x 2*block_size x head_size
        return out

In [7]:
class MultiHead(nn.Module):
    # multiple heads of self-attention
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads*head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # batch_size x 2*block_size x (num_heads*head_size)
        out = self.dropout(self.proj(out)) # batch_size x 2*block_size x n_embd
        return out

In [8]:
class FeedForward(nn.Module):
    # feed forward network
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)  

In [9]:
class Block(nn.Module):
    def __init__(self, n_embd, num_heads):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        head_size = n_embd // num_heads
        self.mh = MultiHead(num_heads, head_size)
        self.ff = FeedForward(n_embd)
    def forward(self,x):
        x = x + self.mh(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

In [ ]:
def train_loss(y_true, y_pred):
    return F.mse_loss(y_true, y_pred)

In [10]:
class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        # exchangeable data so no positional embedding
        self.blocks = nn.Sequential(*[Block(n_embd, num_heads) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embd)
        self.proj_i = nn.Linear(n_dims, n_embd)
        self.proj_f = nn.Linear(n_embd, n_dims)
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
    def forward(self,x):
        x_orig = x.clone()
        x = self.proj_i(x)
        x = self.blocks(x) # batch_size x 2*block_size x n_embd
        x = self.ln(x) # batch_size x 2*block_size x n_embd
        x = self.proj_f(x) # batch_size x 2*block_size x n_dims
        return x[:,::2,0], train_loss(x[:,::2,0], x_orig[:,::2,0])  

In [11]:
model = GPTModel().to(device)
print(sum(p.numel() for p in model.parameters()))
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

6970


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            batch = get_batch(split)
            y_hat, loss = model(batch)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
for iter in range(max_iters):
    # regularly evaluate the loss on train and val 
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    # sample a batch of data
    batch = get_batch('train')
    # forward pass
    y_hat, loss = model(batch)
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [30]:
data_test = get_batch('train')
y_true = data_test[:,-1,0].clone()
data_test[:,-1,0] = 0
y_pred = model(data_test)[0][:,-1]
print(f'model mse : {train_loss(y_true,y_pred).item()}')
# baseline linear regression
X = data_test[:,:-2:2,:]
y = data_test[:,1:-1:2,:1]
X_test = data_test[:,-2:-1,:]
print(f'baseline mse : {train_loss(y_true,(X_test@(torch.pinverse(X.transpose(-1,-2) @ X) @ X.transpose(-1,-2)@y))[:,0,0]).item()}')

model mse : 6.986701011657715
baseline mse : 2.216775374375146e-12
